# RunnableLambda

`RunnableLambda` 는 사용자 정의 함수를 실행 할 수 있는 기능을 제공합니다.

이를 통해 개발자는 자신만의 함수를 정의하고, 해당 함수를 `RunnableLambda` 를 사용하여 실행할 수 있습니다.

예를 들어, 데이터 전처리, 계산, 또는 외부 API와의 상호 작용과 같은 작업을 수행하는 함수를 정의하고 실행할 수 있습니다.

사용자 정의 함수를 실행하는 방법

**주의사항**

사용자 정의 함수를 `RunnableLambda로` 래핑하여 활용할 수 있는데, 여기서 주의할 점은 `사용자 정의 함수가 받을 수 있는 인자는 1개 뿐`이라는 점 입니다.

만약, 여러 인수를 받는 함수로 구현하고 싶다면, 단일 입력을 받아들이고 이를 여러 인수로 풀어내는 래퍼를 작성해야 합니다.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI


In [5]:

def length_function(text):
    return len(text)

def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)

def multiple_length_function(
    _dict 
): # 딕셔너리에서 "text1"과 "text2"의 길이를 곱하는 함수
    return _multiple_length_function(_dict["text1"], _dict["text2"])

In [6]:
# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template("what is {a} + {b}?")
# ChatOpenAI 모델 초기화
model = ChatOpenAI()

In [8]:
# 프롬프트와 모델을 연결하여 체인 생성
chain1 = prompt | model

# 체인 구성
# itemgetter은 딕셔너리 구조에서 key값으로 value값을 꺼낼 때 사용한다.
chain = (
    {
        "a": itemgetter("input_1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("input_1"), "text2": itemgetter("input_2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [9]:
# 주어진 인자들로 체인을 실행합니다.
chain.invoke({"input_1": "bar", "input_2": "gah"})

'3 + 9 = 12'